In [1]:
import os
import time
import pandas as pd
import numpy as np
from geojson import dump
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
import networkx as nx
import os
import osmnx as ox

In [3]:
# Directories paths
modelRunsDir = "data"

# File names (no complete path as they might be present in more folders with the same name)
# e.g. check that this file is in AAA folder, otherwise load it from BBB folder
ZoneCodesFilename = 'EWS_ZoneCodes.csv'

#cost matrix names
QUANTCijRoadMinFilename = 'dis_roads_min.bin'
QUANTCijBusMinFilename = 'dis_bus_min.bin'
QUANTCijRailMinFilename = 'dis_gbrail_min.bin'

QUANTCijRoadMinFilename_OXF = 'Cij_road_min_OXF.bin' # England and Wales
QUANTCijBusMinFilename_OXF = 'Cij_bus_min_OXF.bin' # England and Wales
QUANTCijRailMinFilename_OXF = 'Cij_gbrail_min_OXF.bin' # England and Wales

CijRoadMinFilename = 'Cij_road_min.bin'
CijBusMinFilename = 'Cij_bus_min.bin'
CijRailMinFilename = 'Cij_gbrail_min.bin'

SObsRoadFilename_OXF = 'SObs_1_OXF.bin'
SObsBusFilename_OXF = 'SObs_2_OXF.bin'
SObsRailFilename_OXF = 'SObs_3_OXF.bin'

#centroids for the cost matrices
QUANTCijRoadCentroidsFilename = 'roadcentroidlookup_QC.csv'
QUANTCijBusCentroidsFilename = 'buscentroidlookup_QC.csv'
QUANTCijRailCentroidsFilename = 'gbrailcentroidlookup_QC.csv'

########################################################################################################################
# -- INPUT FILES --

# Retail data
data_open_geolytix_regression_OXF = os.path.join(modelRunsDir,"geolytix_retailpoints_open_regression_OXF.csv")

# Census data
data_census_QS103_OXF = os.path.join(modelRunsDir,"QS103EW_MSOA_OXF.csv")

########################################################################################################################
# -- OUTPUT FILES --

### Journey to work model
# Employment
data_jobs_employment = os.path.join(modelRunsDir,"jobsEmployment.csv")
Jobs_DjOi_2011 = os.path.join(modelRunsDir, "Jobs_DjOi_2011.csv")

# Zones and attractors
data_HH_zones_2011 = os.path.join(modelRunsDir,"jobs_Pop_Zones_2011.csv")

data_HH_attractors_2011 = os.path.join(modelRunsDir,"jobs_HH_Attractors_2011.csv")

# Job accessibility
data_jobs_accessibility_2011 = os.path.join(modelRunsDir,"jobs_accessibility_2011.csv")

# Housing accessibility
data_housing_accessibility_2011 = os.path.join(modelRunsDir,"housing_accessibility_2011.csv")


### Retail (Income) model
# Cost matrices
data_retailpoints_cij_roads = os.path.join(modelRunsDir,"retailpointsCij_roads.bin")
data_retailpoints_cij_bus = os.path.join(modelRunsDir,"retailpointsCij_bus.bin")
data_retailpoints_cij_rail = os.path.join(modelRunsDir,"retailpointsCij_rail.bin")

data_retailpoints_cij_roads_csv = os.path.join(modelRunsDir,"retailpointsCij_roads.csv")
data_retailpoints_cij_bus_csv = os.path.join(modelRunsDir,"retailpointsCij_bus.csv")
data_retailpoints_cij_rail_csv = os.path.join(modelRunsDir,"retailpointsCij_rail.csv")
# Zones and attractors
data_retailpoints_zones = os.path.join(modelRunsDir,"retailpointsZones.csv")
data_retailpoints_attractors = os.path.join(modelRunsDir,"retailpointsAttractors.csv")
# Population
data_retailpoints_population_EWS = os.path.join(modelRunsDir,"retailpointsPopulation_EWS.csv")
data_retailpoints_population_EW = os.path.join(modelRunsDir,"retailpointsPopulation_EW.csv")
data_retailpoints_population_OXF = os.path.join(modelRunsDir,"retailpointsPopulation_OXF.csv")
# Probabilities
data_retailpoints_probRij_roads = os.path.join(modelRunsDir,"retailpointsProbRij_roads.bin")
data_retailpoints_probRij_bus = os.path.join(modelRunsDir,"retailpointsProbRij_bus.bin")
data_retailpoints_probRij_rail = os.path.join(modelRunsDir,"retailpointsProbRij_rail.bin")
# Flows
data_retailpoints_Rij_roads = os.path.join(modelRunsDir,"retailpointsRij_roads.bin")
data_retailpoints_Rij_bus = os.path.join(modelRunsDir,"retailpointsRij_bus.bin")
data_retailpoints_Rij_rail = os.path.join(modelRunsDir,"retailpointsRij_rail.bin")

### Retail (Population) model
# Zones and attractors
data_populationretail_zones = os.path.join(modelRunsDir,"populationretailZones.csv")
data_populationretail_attractors = os.path.join(modelRunsDir,"populationretailAttractors.csv")
# Population
data_populationretail_population_OXF_2011 = os.path.join(modelRunsDir,"populationretailPopulation_OXF_2011.csv")
data_populationretail_population_OXF_2019 = os.path.join(modelRunsDir,"populationretailPopulation_OXF_2019.csv")
data_populationretail_population_OXF_2030 = os.path.join(modelRunsDir,"populationretailPopulation_OXF_2030.csv")
# Probabilities
data_populationretail_probRij_roads_2011_csv = os.path.join(modelRunsDir,"populationretailProbRij_roads_2011.csv")
data_populationretail_probRij_bus_2011_csv = os.path.join(modelRunsDir,"populationretailProbRij_bus_2011.csv")
data_populationretail_probRij_rail_2011_csv = os.path.join(modelRunsDir,"populationretailProbRij_rail_2011.csv")
# Flows
data_populationretail_Rij_roads_2011_csv = os.path.join(modelRunsDir,"populationretailRij_roads_2011.csv")
data_populationretail_Rij_bus_2011_csv = os.path.join(modelRunsDir,"populationretailRij_bus_2011.csv")
data_populationretail_Rij_rail_2011_csv = os.path.join(modelRunsDir,"populationretailRij_rail_2011.csv")

# Population
data_totalpopulation = os.path.join(modelRunsDir,"totalpopulation_englandwalesscotland_msoaiz.csv") #this is QS103 col All People joined for E+W+S
data_agepopulation = os.path.join(modelRunsDir,"agepopulation_englandwalesscotland_msoaiz.csv") #same as total pop, but with age breakdown

In [ ]:
# The keys of the inputs and outputs dictionaries, as well as the file names should follow the camelCase notation.

inputs = {}

inputs["DataOpenGeolytixRegression"] = "data/geolytix_retailpoints_open_regression.csv"
inputs["DataCensusQS103"]            = "data/QS103EW_MSOA.csv"
inputs["DataCensusQS103SC"]          = "data/QS103SC_DZ2001.csv"
inputs["LookupDZ2001toIZ2001"]       = "data/DZ2001Lookup.csv"
inputs["OxfMsoaFile"]                = "data/OXF_MSOA.csv"
inputs["ZonesCoordinates"]           = "data/zones_data_coordinates.csv"
inputs["QUANTCijRoadMinFilename"]    = "data/QUANT_matrices/dis_roads_min.bin"
inputs["QUANTCijBusMinFilename"]     = "data/QUANT_matrices/dis_bus_min.bin"
inputs["QUANTCijRailMinFilename"]    = "data/dis_gbrail_min.bin"
inputs["SObsRoadFilename"]           = "data/SObs_1.bin"
inputs["SObsBusFilename"]            = "data/SObs_2.bin"
inputs["SObsRailFilename"]           = "data/SObs_3.bin"
inputs["OXFNewHousingDev"]           = "data/New_housing_dev_table.csv"

inputs["Employment2011GB"]          = "./HARMONY_LUTI_OXF/external-data/Employment/Employment_by_sector_GB_MSOA.csv"
inputs["Employment2019"]            = "./HARMONY_LUTI_OXF/external-data/Employment/Employment_2019.csv"
inputs["Employment2030"]            = "./HARMONY_LUTI_OXF/external-data/Employment/Employment_2030.csv"

inputs["DwellingsOxf"]               = "./HARMONY_LUTI_OXF/external-data/Housing_attractiveness/OXF-Dwellings2011.csv"
inputs["ZonesCoordinates"]           = "./HARMONY_LUTI_OXF/external-data/zones_data_coordinates.csv"
inputs["OxfPostcodes"]               = "./HARMONY_LUTI_OXF/external-data/Oxfordshire_postcodes.csv"
inputs["DataSchoolsEWSPrimary"]      = "./HARMONY_LUTI_OXF/external-data/OSF_RAMPUrbanAnalytics/primary_ews.csv"
inputs["DataSchoolsEWSPSecondary"]   = "./HARMONY_LUTI_OXF/external-data/OSF_RAMPUrbanAnalytics/secondary_ews.csv"
inputs["DataHospitals"]              = "./HARMONY_LUTI_OXF/external-data/OSF_RAMPUrbanAnalytics/NHS Estates Information/NHS_join_mod.csv"
inputs["MsoaShapefile"]              = "./HARMONY_LUTI_OXF/external-data/geography/msoa_OXF.shp"
inputs["RoadNetworkShapefile"]       = "./HARMONY_LUTI_OXF/external-data/geography/Major_Roads_OXF.shp"
inputs["MSOACentroidsShapefile"]     = "./HARMONY_LUTI_OXF/external-data/geography/OXF_MSOA_centroids.shp"
inputs["MSOACentroidsShapefileWGS84"]     = "./HARMONY_LUTI_OXF/external-data/geography/OXF_MSOA_centroids_WGS84.shp"

outputs = {}

outputs["JobsAccessibility2019"]            = "./Outputs-Oxfordshire/jobs_accessibility_2019.csv"
outputs["JobsAccessibility2030"]            = "./Outputs-Oxfordshire/jobs_accessibility_2030.csv"
outputs["HousingAccessibility2019"]         = "./Outputs-Oxfordshire/housing_accessibility_2019.csv"
outputs["HousingAccessibility2030"]         = "./Outputs-Oxfordshire/housing_accessibility_2030.csv"
outputs["JobsDjOi2019"]                     = "./Outputs-Oxfordshire/Jobs_DjOi_2019.csv"
outputs["JobsDjOi2030"]                     = "./Outputs-Oxfordshire/Jobs_DjOi_2030.csv"
outputs["JobsProbTijRoads2019"]             = "./Outputs-Oxfordshire/jobsProbTij_roads_2019.csv"
outputs["JobsProbTijBus2019"]               = "./Outputs-Oxfordshire/jobsProbTij_bus_2019.csv"
outputs["JobsProbTijRail2019"]              = "./Outputs-Oxfordshire/jobsProbTij_rail_2019.csv"
outputs["JobsTijRoads2019"]                 = "./Outputs-Oxfordshire/jobsTij_roads_2019.csv"
outputs["JobsTijBus2019"]                   = "./Outputs-Oxfordshire/jobsTij_bus_2019.csv"
outputs["JobsTijRail2019"]                  = "./Outputs-Oxfordshire/jobsTij_rail_2019.csv"
outputs["ArrowsFlowsCar2019"]               = "./Outputs-Oxfordshire/flows_2019_car.geojson"
outputs["ArrowsFlowsBus2019"]               = "./Outputs-Oxfordshire/flows_2019_bus.geojson"
outputs["ArrowsFlowsRail2019"]              = "./Outputs-Oxfordshire/flows_2019_rail.geojson"
outputs["JobsProbTijRoads2030"]             = "./Outputs-Oxfordshire/jobsProbTij_roads_2030.csv"
outputs["JobsProbTijBus2030"]               = "./Outputs-Oxfordshire/jobsProbTij_bus_2030.csv"
outputs["JobsProbTijRail2030"]              = "./Outputs-Oxfordshire/jobsProbTij_rail_2030.csv"
outputs["JobsTijRoads2030"]                 = "./Outputs-Oxfordshire/jobsTij_roads_2030.csv"
outputs["JobsTijBus2030"]                   = "./Outputs-Oxfordshire/jobsTij_bus_2030.csv"
outputs["JobsTijRail2030"]                  = "./Outputs-Oxfordshire/jobsTij_rail_2030.csv"
outputs["ArrowsFlowsCar2030"]               = "./Outputs-Oxfordshire/flows_2030_car.geojson"
outputs["ArrowsFlowsBus2030"]               = "./Outputs-Oxfordshire/flows_2030_bus.geojson"
outputs["ArrowsFlowsRail2030"]              = "./Outputs-Oxfordshire/flows_2030_rail.geojson"
outputs["RetailProbRijRoads2019"]           = "./Outputs-Oxfordshire/retailProbRij_roads_2019.csv"
outputs["RetailProbRijBus2019"]             = "./Outputs-Oxfordshire/retailProbRij_bus_2019.csv"
outputs["RetailProbRijRail2019"]            = "./Outputs-Oxfordshire/retailProbRij_rail_2019.csv"
outputs["RetailRijRoads2019"]               = "./Outputs-Oxfordshire/retailRij_roads_2019.csv"
outputs["RetailRijBus2019"]                 = "./Outputs-Oxfordshire/retailRij_bus_2019.csv"
outputs["RetailRijRail2019"]                = "./Outputs-Oxfordshire/retailRij_rail_2019.csv"
outputs["RetailProbRijRoads2030"]           = "./Outputs-Oxfordshire/retailProbRij_roads_2030.csv"
outputs["RetailProbRijBus2030"]             = "./Outputs-Oxfordshire/retailProbRij_bus_2030.csv"
outputs["RetailProbRijRail2030"]            = "./Outputs-Oxfordshire/retailProbRij_rail_2030.csv"
outputs["RetailRijRoads2030"]               = "./Outputs-Oxfordshire/retailRij_roads_2030.csv"
outputs["RetailRijBus2030"]                 = "./Outputs-Oxfordshire/retailRij_bus_2030.csv"
outputs["RetailRijRail2030"]                = "./Outputs-Oxfordshire/retailRij_rail_2030.csv"
outputs["PrimaryProbPijRoads2019"]          = "./Outputs-Oxfordshire/primaryProbPij_roads_2019.csv"
outputs["PrimaryProbPijBus2019"]            = "./Outputs-Oxfordshire/primaryProbPij_bus_2019.csv"
outputs["PrimaryProbPijRail2019"]           = "./Outputs-Oxfordshire/primaryProbPij_rail_2019.csv"
outputs["PrimaryPijRoads2019"]              = "./Outputs-Oxfordshire/primaryPij_roads_2019.csv"
outputs["PrimaryPijBus2019"]                = "./Outputs-Oxfordshire/primaryPij_bus_2019.csv"
outputs["PrimaryPijRail2019"]               = "./Outputs-Oxfordshire/primaryPij_rail_2019.csv"
outputs["PrimaryProbPijRoads2030"]          = "./Outputs-Oxfordshire/primaryProbPij_roads_2030.csv"
outputs["PrimaryProbPijBus2030"]            = "./Outputs-Oxfordshire/primaryProbPij_bus_2030.csv"
outputs["PrimaryProbPijRail2030"]           = "./Outputs-Oxfordshire/primaryProbPij_rail_2030.csv"
outputs["PrimaryPijRoads2030"]              = "./Outputs-Oxfordshire/primaryPij_roads_2030.csv"
outputs["PrimaryPijBus2030"]                = "./Outputs-Oxfordshire/primaryPij_bus_2030.csv"
outputs["PrimaryPijRail2030"]               = "./Outputs-Oxfordshire/primaryPij_rail_2030.csv"
outputs["SecondaryProbPijRoads2019"]        = "./Outputs-Oxfordshire/secondaryProbPij_roads_2019.csv"
outputs["SecondaryProbPijBus2019"]          = "./Outputs-Oxfordshire/secondaryProbPij_bus_2019.csv"
outputs["SecondaryProbPijRail2019"]         = "./Outputs-Oxfordshire/secondaryProbPij_rail_2019.csv"
outputs["SecondaryPijRoads2019"]            = "./Outputs-Oxfordshire/secondaryPij_roads_2019.csv"
outputs["SecondaryPijBus2019"]              = "./Outputs-Oxfordshire/secondaryPij_bus_2019.csv"
outputs["SecondaryPijRail2019"]             = "./Outputs-Oxfordshire/secondaryPij_rail_2019.csv"
outputs["SecondaryProbPijRoads2030"]        = "./Outputs-Oxfordshire/secondaryProbPij_roads_2030.csv"
outputs["SecondaryProbPijBus2030"]          = "./Outputs-Oxfordshire/secondaryProbPij_bus_2030.csv"
outputs["SecondaryProbPijRail2030"]         = "./Outputs-Oxfordshire/secondaryProbPij_rail_2030.csv"
outputs["SecondaryPijRoads2030"]            = "./Outputs-Oxfordshire/secondaryPij_roads_2030.csv"
outputs["SecondaryPijBus2030"]              = "./Outputs-Oxfordshire/secondaryPij_bus_2030.csv"
outputs["SecondaryPijRail2030"]             = "./Outputs-Oxfordshire/secondaryPij_rail_2030.csv"
outputs["HospitalProbHijRoads2019"]         = "./Outputs-Oxfordshire/hospitalProbHij_roads_2019.csv"
outputs["HospitalProbHijBus2019"]           = "./Outputs-Oxfordshire/hospitalProbHij_bus_2019.csv"
outputs["HospitalProbHijRail2019"]          = "./Outputs-Oxfordshire/hospitalProbHij_rail_2019.csv"
outputs["HospitalHijRoads2019"]             = "./Outputs-Oxfordshire/hospitalHij_roads_2019.csv"
outputs["HospitalHijBus2019"]               = "./Outputs-Oxfordshire/hospitalHij_bus_2019.csv"
outputs["HospitalHijRail2019"]              = "./Outputs-Oxfordshire/hospitalHij_rail_2019.csv"
outputs["HospitalProbHijRoads2030"]         = "./Outputs-Oxfordshire/hospitalProbHij_roads_2030.csv"
outputs["HospitalProbHijBus2030"]           = "./Outputs-Oxfordshire/hospitalProbHij_bus_2030.csv"
outputs["HospitalProbHijRail2030"]          = "./Outputs-Oxfordshire/hospitalProbHij_rail_2030.csv"
outputs["HospitalHijRoads2030"]             = "./Outputs-Oxfordshire/hospitalHij_roads_2030.csv"
outputs["HospitalHijBus2030"]               = "./Outputs-Oxfordshire/hospitalHij_bus_2030.csv"
outputs["HospitalHijRail2030"]              = "./Outputs-Oxfordshire/hospitalHij_rail_2030.csv"
outputs["MapPopChange20192030"]             = "./Outputs-Oxfordshire/pop_change_19-30.png"
outputs["MapHousingAccChange20192030Roads"] = "./Outputs-Oxfordshire/HA_change_19-30_roads.png"
outputs["MapHousingAccChange20192030Bus"]   = "./Outputs-Oxfordshire/HA_change_19-30_bus.png"
outputs["MapHousingAccChange20192030Rail"]  = "./Outputs-Oxfordshire/HA_change_19-30_rail.png"
outputs["MapJobsAccChange20192030Roads"]    = "./Outputs-Oxfordshire/JA_change_19-30_roads.png"
outputs["MapJobsAccChange20192030Bus"]      = "./Outputs-Oxfordshire/JA_change_19-30_bus.png"
outputs["MapJobsAccChange20192030Rail"]     = "./Outputs-Oxfordshire/JA_change_19-30_rail.png"
outputs["MapResultsShapefile"]              = "./Outputs-Oxfordshire/OXF_results.shp"
outputs["JobsTijRoads2019FlowMap"]          = "./Outputs-Oxfordshire/JobsTijRoads2019FlowMap.png"
outputs["JobsTijBus2019FlowMap"]            = "./Outputs-Oxfordshire/JobsTijBus2019FlowMap.png"
outputs["JobsTijRail2019FlowMap"]           = "./Outputs-Oxfordshire/JobsTijRail2019FlowMap.png"
outputs["JobsTijRoads2030FlowMap"]          = "./Outputs-Oxfordshire/JobsTijRoads2030FlowMap.png"
outputs["JobsTijBus2030FlowMap"]            = "./Outputs-Oxfordshire/JobsTijBus2030FlowMap.png"
outputs["JobsTijRail2030FlowMap"]           = "./Outputs-Oxfordshire/JobsTijRail2030FlowMap.png"

In [4]:
def buildTotalPopulationTable(censusEW, censusSC, lookupSC):
    # England and Wales
    dfEW = pd.read_csv(censusEW)
    dfEW['count_allpeople'] = dfEW['Age: All categories: Age; measures: Value'] # you could just rename the col, not copy
    dfEW2 = pd.DataFrame({'geography code': dfEW['geography code'], 'count_allpeople': dfEW['count_allpeople']})
    
    # Scotland (DZ2001)
    dfS = pd.read_csv(censusSC) # join on "Unnamed: 0", it's blank! This is the datazone code field
    dfS.set_index('Unnamed: 0')
    dfSLookup = pd.read_csv(lookupSC) # join on ZONECODE, which is the datazone code
    dfS = dfS.join(other=dfSLookup.set_index('ZONECODE'),on='Unnamed: 0')
    dfS['count_allpeople'] = dfS['All people']
    dfS2 = dfS.groupby(['IZ_CODE']).agg({'count_allpeople': "sum"})
    dfS3 = pd.DataFrame({'msoaiz': dfS2.index, 'count_allpeople': dfS2['count_allpeople'] }) # cols are weird if you don't do this - dfS2 has iz as the index and we need a col
    # NOTE: I deleted the line in the Scotland QS103SC file which contains the total for the whole of Scotland - it's a pain

    # Now merge dfEW2 and dfS3 into one table
    dfEW2.reset_index()
    dfS3.reset_index()
    dfEW2.columns=['msoaiz','count_allpeople']
    dfEWS = dfEW2.append(dfS3)
    dfEWS.to_csv(data_totalpopulation,index=False) # drop the index col off as EW has a numberic row id and scot has idx=IZ code - why on earth??

    # Now repeat, but adding in the age data to make an age structure table
    # Scotland is a problem... we have to aggregate all these columns by IZ groups of DZ entries
    # ,All people,Under 1,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100 and over
    dfSA = dfS.groupby(['IZ_CODE']).sum()
    dfSA.index.rename("msoaiz", inplace=True)
    dfSA['msoaiz']=dfSA.index #have to copy index into regular field, otherwise I can't merge it with the EW data
    dfSA.drop({'EASTING', 'NORTHING', 'MMW_CODE', 'SPC07_CODE', 'LA_CODE','SGUR2010_6FOLD', 'LEC_CODE', 'SETT2008_CODE'},axis=1,inplace=True) #and drop the Scotland specific columns
    # dfSA.to_csv(data_agepopulation,index=True)
    # now you need to merge in England and Wales
    dfEW.reset_index()
    dfEW.drop({'Rural Urban'},axis=1,inplace=True) # drop this col as not in Scotland data - what is it anyway, all it contains is 'Total'?
    dfEW.rename({'geography code': 'msoaiz'},axis=1,inplace=True)
    dfEW.rename({'Age: All categories: Age; measures: Value': 'All people'},axis=1,inplace=True)
    dfEW.rename({'Age: Age under 1; measures: Value': 'Under 1'},axis=1,inplace=True)
    for i in range(1,100):
        dfEW.rename({'Age: Age '+str(i)+'; measures: Value': str(i)},axis=1,inplace=True)
    dfEW.rename({'Age: Age 100 and over; measures: Value': '100 and over'},axis=1,inplace=True)
    # That gets all the age column names the same, now we can append them together NOTE: there are some column differences between the two data
    dfAgeEWS = dfEW.append(dfSA)
    dfAgeEWS.to_csv(data_agepopulation,index=False)

In [ ]:
def start_main(inputs, outputs):
    ################################################################################
    # Initialisation                                                               #
    ################################################################################

    # NOTE: this section provides the base data for the models that come later. This
    # will only be run on the first run of the program to assemble all the tables
    # required from the original sources. After that, if the file exists in the
    # directory, then nothing new is created and this section is effectively
    # skipped, up until the model run section at the end.

    # make a model-runs dir if we need it
    if not os.path.exists(modelRunsDir):
        os.makedirs(modelRunsDir)

    # Downloads first:

    ################################################################################
    # File creation

    # same thing for the total population of England, Scotland and Wales - needed for hospitals
    # now join total population for England/Wales and Scotland files together
    if not os.path.isfile(data_totalpopulation):
        buildTotalPopulationTable(inputs["DataCensusQS103"], inputs["DataCensusQS103SC"], inputs["LookupDZ2001toIZ2001"])

    ################################################################################
    # Model run section
    ################################################################################
    print()
    print("Importing QUANT cij matrices")

    OXF_MSOA_df =  pd.read_csv(inputs["OxfMsoaFile"], usecols=["msoa11cd"]) # import file with OXF MSOA codes
    OXF_MSOA_df.columns = ['areakey'] # rename column "msoa11cd" to "areakey"
    OXF_MSOA_list = OXF_MSOA_df['areakey'].tolist()

    # load zone codes lookup file to convert MSOA codes into zone i indexes for the model
    zonecodes_EWS = pd.read_csv(os.path.join(modelRunsDir,ZoneCodesFilename))
    zonecodes_EWS.set_index('areakey')
    zonecodes_EWS_list = zonecodes_EWS['areakey'].tolist()

    #_____________________________________________________________________________________
    # IMPORT cij QUANT matrices
    # ROADS cij
    print()
    print("Importing QUANT roads cij for Oxfordshire")

    if not os.path.isfile(os.path.join(modelRunsDir,QUANTCijRoadMinFilename_OXF)):
        # load cost matrix, time in minutes between MSOA zones for roads:
        cij_road_EWS = loadQUANTMatrix(inputs["QUANTCijRoadMinFilename"])
        cij_road_EWS_df = pd.DataFrame(cij_road_EWS, index=zonecodes_EWS_list, columns=zonecodes_EWS_list) # turn the numpy array into a pd dataframe, (index and columns: MSOA codes)
        cij_road_OXF_df = cij_road_EWS_df[OXF_MSOA_list]  # Create OXF df filtering EWS columns
        cij_road_OXF_df = cij_road_OXF_df.loc[OXF_MSOA_list]  # Filter rows
        cij_road_OXF = cij_road_OXF_df.to_numpy()  # numpy matrix for OXF (same format as utils loadQUANTMatrix)
        cij_road_OXF[cij_road_OXF < 1] = 1  # lower limit of 1 minute links
        saveMatrix(cij_road_OXF, os.path.join(modelRunsDir,QUANTCijRoadMinFilename_OXF))
        # save as csv file
        np.savetxt(os.path.join(modelRunsDir, "cij_road_OXF.csv"), cij_road_OXF, delimiter=",")
    else:
        cij_road_OXF = loadMatrix(os.path.join(modelRunsDir,QUANTCijRoadMinFilename_OXF))
        print('cij roads shape: ', cij_road_OXF.shape)

    # Export cij matrices for checking
    # np.savetxt(os.path.join(modelRunsDir,'debug_cij_roads.csv'), cij_road_OXF, delimiter=',', fmt='%i')

    #_____________________________________________________________________________________
    # BUS & FERRIES cij
    print()
    print("Importing QUANT bus cij for Oxfordshire")

    if not os.path.isfile(os.path.join(modelRunsDir,QUANTCijBusMinFilename_OXF)):
        # load cost matrix, time in minutes between MSOA zones for bus and ferries network:
        cij_bus_EWS = loadQUANTMatrix(inputs["QUANTCijBusMinFilename"])
        cij_bus_EWS_df = pd.DataFrame(cij_bus_EWS, index=zonecodes_EWS_list, columns=zonecodes_EWS_list) # turn the numpy array into a pd dataframe, (index and columns: MSOA codes)
        cij_bus_OXF_df = cij_bus_EWS_df[OXF_MSOA_list]  # Create OXF df filtering EWS columns
        cij_bus_OXF_df = cij_bus_OXF_df.loc[OXF_MSOA_list]  # Filter rows
        cij_bus_OXF = cij_bus_OXF_df.to_numpy()  # numpy matrix for OXF (same format as utils loadQUANTMatrix)
        cij_bus_OXF[cij_bus_OXF < 1] = 1  # lower limit of 1 minute links
        saveMatrix(cij_bus_OXF, os.path.join(modelRunsDir, QUANTCijBusMinFilename_OXF))
        # save as csv file
        np.savetxt(os.path.join(modelRunsDir, "cij_bus_OXF.csv"), cij_bus_OXF, delimiter=",")
    else:
        cij_bus_OXF = loadMatrix(os.path.join(modelRunsDir,QUANTCijBusMinFilename_OXF))
        print('cij bus shape: ', cij_bus_OXF.shape)

    # Export cij matrices for checking
    # np.savetxt(os.path.join(modelRunsDir,'debug_cij_bus.csv'), cij_bus_OXF, delimiter=',', fmt='%i')

    #_____________________________________________________________________________________
    # RAILWAYS cij
    print()
    print("Importing QUANT rail cij for Oxfordshire")

    if not os.path.isfile(os.path.join(modelRunsDir,QUANTCijRailMinFilename_OXF)):
        # load cost matrix, time in minutes between MSOA zones for railways:
        cij_rail_EWS = loadQUANTMatrix(inputs["QUANTCijRailMinFilename"])
        cij_rail_EWS_df = pd.DataFrame(cij_rail_EWS, index=zonecodes_EWS_list, columns=zonecodes_EWS_list) # turn the numpy array into a pd dataframe, (index and columns: MSOA codes)
        cij_rail_OXF_df = cij_rail_EWS_df[OXF_MSOA_list]  # Create OXF df filtering EWS columns
        cij_rail_OXF_df = cij_rail_OXF_df.loc[OXF_MSOA_list]  # Filter rows
        cij_rail_OXF = cij_rail_OXF_df.to_numpy()  # numpy matrix for OXF (same format as utils loadQUANTMatrix)
        cij_rail_OXF[cij_rail_OXF < 1] = 1  # lower limit of 1 minute links
        saveMatrix(cij_rail_OXF, os.path.join(modelRunsDir, QUANTCijRailMinFilename_OXF))
        # save as csv file
        np.savetxt(os.path.join(modelRunsDir, "cij_rail_OXF.csv"), cij_rail_OXF, delimiter=",")
    else:
        cij_rail_OXF = loadMatrix(os.path.join(modelRunsDir,QUANTCijRailMinFilename_OXF))
        print('cij rail shape: ', cij_rail_OXF.shape)

    # Export cij matrices for checking
    # np.savetxt(os.path.join(modelRunsDir,'debug_cij_rail.csv'), cij_rail_OXF, delimiter=',', fmt='%i')

    print()
    print("Importing QUANT cij matrices completed.")
    print()
    #_____________________________________________________________________________________

    # IMPORT SObs QUANT matrices: observed trips
    print("Importing SObs matrices")

    # SObs ROADS
    print()
    print("Importing SObs for roads for Oxfordshire")

    if not os.path.isfile(os.path.join(modelRunsDir,SObsRoadFilename_OXF)):
        # load observed trips matrix for roads:
        SObs_road_EWS = loadQUANTMatrix(inputs["SObsRoadFilename"])
        SObs_road_EWS_df = pd.DataFrame(SObs_road_EWS, index=zonecodes_EWS_list, columns=zonecodes_EWS_list) # turn the numpy array into a pd dataframe, (index and columns: MSOA codes)
        SObs_road_OXF_df = SObs_road_EWS_df[OXF_MSOA_list]  # Create OXF df filtering EWS columns
        SObs_road_OXF_df = SObs_road_OXF_df.loc[OXF_MSOA_list]  # Filter rows
        SObs_road_OXF = SObs_road_OXF_df.to_numpy()  # numpy matrix for OXF (same format as utils loadQUANTMatrix)
        saveMatrix(SObs_road_OXF, os.path.join(modelRunsDir, SObsRoadFilename_OXF))
    # else:
    #     SObs_road_OXF = loadMatrix(os.path.join(modelRunsDir,SObsRoadFilename_OXF))
    #     print('Sobs road shape: ', SObs_road_OXF.shape)
    #_____________________________________________________________________________________

    # SObs BUS & FERRIES
    print()
    print("Importing SObs for bus & ferries for Oxfordshire")

    if not os.path.isfile(os.path.join(modelRunsDir,SObsBusFilename_OXF)):
        # load observed trips matrix for bus and ferries:
        SObs_bus_EWS = loadQUANTMatrix(inputs["SObsBusFilename"])
        SObs_bus_EWS_df = pd.DataFrame(SObs_bus_EWS, index=zonecodes_EWS_list, columns=zonecodes_EWS_list)  # turn the numpy array into a pd dataframe, (index and columns: MSOA codes)
        SObs_bus_OXF_df = SObs_bus_EWS_df[OXF_MSOA_list]  # Create OXF df filtering EWS columns
        SObs_bus_OXF_df = SObs_bus_OXF_df.loc[OXF_MSOA_list]  # Filter rows
        SObs_bus_OXF = SObs_bus_OXF_df.to_numpy()  # numpy matrix for OXF (same format as utils loadQUANTMatrix)
        saveMatrix(SObs_bus_OXF, os.path.join(modelRunsDir,SObsBusFilename_OXF))
    # else:
    #     SObs_bus_OXF = loadMatrix(os.path.join(modelRunsDir,SObsBusFilename_OXF))
    #     print('Sobs bus shape: ', SObs_bus_OXF.shape)
    #_____________________________________________________________________________________

    # SObs RAIL
    print()
    print("Importing SObs for rail for Oxfordshire")

    if not os.path.isfile(os.path.join(modelRunsDir,SObsRailFilename_OXF)):
        # load observed trips matrix for rails:
        SObs_rail_EWS = loadQUANTMatrix(inputs["SObsRailFilename"])
        SObs_rail_EWS_df = pd.DataFrame(SObs_rail_EWS, index=zonecodes_EWS_list, columns=zonecodes_EWS_list)  # turn the numpy array into a pd dataframe, (index and columns: MSOA codes)
        SObs_rail_OXF_df = SObs_rail_EWS_df[OXF_MSOA_list]  # Create OXF df filtering EWS columns
        SObs_rail_OXF_df = SObs_rail_OXF_df.loc[OXF_MSOA_list]  # Filter rows
        SObs_rail_OXF = SObs_rail_OXF_df.to_numpy()  # numpy matrix for OXF (same format as utils loadQUANTMatrix)
        saveMatrix(SObs_rail_OXF, os.path.join(modelRunsDir,SObsRailFilename_OXF))